In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from typing import Tuple,Dict
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Merge Data

The input data currently contains multiple csv files 
To merge, we backfill (fill NaN with the next value in the col) the columns with lesser entries and create a df for each team.

In [3]:
# create dataframe for each dataset of all teams

teams = ["Digilog","AICU","IUACAAS","Automatoes","TheAutomators"]
datasets = ["GreenhouseClimate.csv","Production.csv"]

df_raw = []
df_teams  = []
i = 0
for i in range(len(teams)):
    df_raw.append([])
    for data in datasets:
        df_tmp = pd.read_csv("/kaggle/input/autonomous-greenhouse-challengeagc-2nd-2019/{}/{}".format(teams[i],data))
        df_tmp = df_tmp.rename(columns={"%Time":"%time","%Time ":"%time"}) # rename Time col to have the same name "%time"
        df_raw[i].append(df_tmp)
    
    df_team = pd.concat(df_raw[i],sort=True,ignore_index=True)  # merge all the datasets
    df_team = df_team.fillna(method="bfill")  # bfill all na fields
    df_team = df_team.fillna(method="ffill")  # ffill all the rest of the fields
    df_team = df_team.fillna(0)  # ffill all the rest of the fields
    
    
    # TODO: ffill the values after bfill 
    
    df_teams.append(df_team)



FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/autonomous-greenhouse-challengeagc-2nd-2019/Digilog/GreenhouseClimate.csv'

In [98]:
df_teams[0].isna().sum()

%Juice                         0
%time                          0
Acid                           0
AssimLight                     0
Bite                           0
                              ..
water_sup                      0
water_sup_intervals_sp_min     0
water_sup_intervals_vip_min    0
window_pos_lee_sp              0
window_pos_lee_vip             0
Length: 120, dtype: int64

In [99]:
# save the merged team csv file
for i in range(len(df_teams)):
    df_teams[i].to_csv("team_{}.csv".format(i),index=False)

## Observations, Actions and Rewards 

In [100]:
for col in df_teams[0].columns:
    print('"{}",'.format(col))

"%Juice",
"%time",
"Acid",
"AssimLight",
"Bite",
"BlackScr",
"CO2_cons",
"CO2air",
"Cum_irr",
"Cum_trusses",
"DMC_fruit",
"Drain",
"EC_drain_PC",
"EC_slab1",
"EC_slab2",
"ElecHigh",
"ElecLow",
"EnScr",
"Flavour ",
"Heat_cons",
"HumDef",
"Irr",
"Nr_fruits_ClassA",
"Nr_fruits_ClassB",
"PipeGrow",
"PipeLow",
"ProdA",
"ProdB",
"Rhair",
"Stem_elong",
"Stem_thick",
"TSS",
"Tair",
"Tot_PAR",
"Tot_PAR_Lamps",
"Truss development time ",
"VentLee",
"Ventwind",
"WC_slab1",
"WC_slab2",
"Weight",
"Weight_fruits_ClassA",
"Weight_fruits_ClassB",
"assim_sp",
"assim_vip",
"avg_nr_harvested_trusses",
"co2_dos",
"co2_sp",
"co2_vip",
"drain_B",
"drain_Ca",
"drain_Cl",
"drain_Cu",
"drain_EC",
"drain_Fe",
"drain_HCO3",
"drain_K",
"drain_Mg",
"drain_Mn",
"drain_Mo",
"drain_NH4",
"drain_NO3",
"drain_Na",
"drain_PH",
"drain_PO4",
"drain_SO4",
"drain_Si",
"drain_Zn",
"dx_sp",
"dx_vip",
"int_blue_sp",
"int_blue_vip",
"int_farred_sp",
"int_farred_vip",
"int_red_sp",
"int_red_vip",
"int_white_sp",
"int_white_vip",

In [101]:
# list of observation, action and reward columns 
observation_cols = ["%Juice","%time","Acid","AssimLight",
                    "Bite","BlackScr","CO2_cons","CO2air",
                    "Cum_irr","Cum_trusses","DMC_fruit","Drain",
                    "EC_drain_PC","EC_slab1","EC_slab2","ElecHigh",
                    "ElecLow","EnScr","Flavour ","Heat_cons",
                    "HumDef","Irr","Nr_fruits_ClassA","Nr_fruits_ClassB",
                    "PipeGrow","PipeLow","ProdA","ProdB",
                    "Rhair","Stem_elong","Stem_thick","TSS",
                    "Tair","Tot_PAR","Tot_PAR_Lamps","Truss development time ",
                    "VentLee","Ventwind","WC_slab1","WC_slab2",
                    "Weight","Weight_fruits_ClassA","Weight_fruits_ClassB","avg_nr_harvested_trusses",
                    "co2_dos","water_sup","t_slab1","t_slab2",
                    "pH_drain_PC","plant_dens","stem_dens ",]

setpoint_columns = ["assim_sp","co2_sp","dx_sp","int_blue_sp","int_farred_sp", \
        "int_red_sp","int_white_sp","scr_blck_sp","scr_enrg_sp","t_grow_min_sp",\
        "t_heat_sp","t_rail_min_sp","t_vent_sp","water_sup_intervals_sp_min",\
        "window_pos_lee_sp"]

reward_cols = ["Weight","Weight_fruits_ClassA","Weight_fruits_ClassB","avg_nr_harvested_trusses",\
              "Nr_fruits_ClassA","Nr_fruits_ClassB","ProdA","ProdB",
               "%Juice","Acid","Bite","Flavour ",
               "TSS","Weight","DMC_fruit",]



In [102]:
# creating reward dataframes for each team
df_rewards = []
for df_team in df_teams:
    df_reward = df_team[reward_cols+["%time"]]
    # accumulating the reward_cols to use it as reward
    df_reward["sum"] = df_reward[reward_cols].sum(axis=1)
    # standardizing the rewards 
    df_reward["sum"] = df_reward["sum"] # TODO: divide it with the max of all teams 
    # appending the reward and time to df
    df_rewards.append(df_reward[["%time","sum"]])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [103]:
# understanding the reward distribution in each team
for df_reward in df_rewards:
    print(df_reward.describe())

              %time           sum
count  95848.000000  95848.000000
mean   43898.010621    481.134194
std       47.919487     71.628025
min    43815.000000    477.920000
25%    43856.513890    477.920000
50%    43898.015625    477.920000
75%    43939.507810    477.920000
max    43981.000000   3283.290000
              %time           sum
count  95849.000000  95849.000000
mean   43898.010381    774.992825
std       47.919295    116.355629
min    43815.000000    769.820000
25%    43856.513890    769.820000
50%    43898.013890    769.820000
75%    43939.506940    769.820000
max    43981.000000   3562.710000
              %time           sum
count  95848.000000  95848.000000
mean   43898.010621    386.024726
std       47.919487     98.652040
min    43815.000000    381.610000
25%    43856.513890    381.610000
50%    43898.015625    381.610000
75%    43939.507810    381.610000
max    43981.000000   2760.713200
              %time           sum
count  95848.000000  95848.000000
mean   43898.0

In [104]:
# creating observation dataframes for each teams
df_observations = []
for df_team in df_teams:
    df_observations.append(df_team[observation_cols])
    
# creating actions dataframe for each teams 
df_actions = []
for df_team in df_teams:
    df_actions.append(df_team[setpoint_columns])

In [130]:
def get_values(team:int,index:int)->Dict[str,pd.Series]:
    """get_value(team,index) -> (observation_current,action,rewards,observation_next)
    """
    observation_current = df_observations[team].iloc[index]
    try:
        observation_next = df_observations[team].iloc[index+1]
    except:
        observation_next = 0
    
    action = df_actions[team].iloc[index]
    rewards = df_rewards[team].iloc[index,1] # get the sum col
    return {"observation_current":observation_current, "action":action,"rewards":rewards,"observation_next":observation_next}

In [131]:
get_values(0,0)

{'observation_current': %Juice                          68.0
 %time                        43836.0
 Acid                            13.3
 AssimLight                         0
 Bite                           193.0
 BlackScr                          35
 CO2_cons                    0.006253
 CO2air                           474
 Cum_irr                          0.2
 Cum_trusses                      1.0
 DMC_fruit                       8.77
 Drain                            0.0
 EC_drain_PC                      1.9
 EC_slab1                         NaN
 EC_slab2                         NaN
 ElecHigh                    0.959833
 ElecLow                          0.0
 EnScr                             96
 Flavour                         78.0
 Heat_cons                     3.2838
 HumDef                           7.1
 Irr                              0.0
 Nr_fruits_ClassA                 0.0
 Nr_fruits_ClassB                 0.0
 PipeGrow                        71.0
 PipeLow                   

In [105]:
df_observations[0]

,%Juice,%time,Acid,AssimLight,Bite,BlackScr,CO2_cons,CO2air,Cum_irr,Cum_trusses,...,Weight_fruits_ClassA,Weight_fruits_ClassB,avg_nr_harvested_trusses,co2_dos,water_sup,t_slab1,t_slab2,pH_drain_PC,plant_dens,stem_dens
0,68.0,43836.0,13.3,0,193.0,35,0.006253,474,0.2,1.0,...,70.0,0.0,0.1,NaN,2.0,NaN,NaN,7.3,1.3,2.6
1,68.0,43850.0,13.3,0,193.0,35,0.006253,474,0.2,1.0,...,70.0,0.0,0.1,NaN,2.0,NaN,NaN,7.3,1.3,2.6
2,68.0,43864.0,13.3,0,193.0,35,0.006253,474,0.2,1.0,...,70.0,0.0,0.1,NaN,2.0,NaN,NaN,7.3,1.3,2.6
3,68.0,43879.0,13.3,0,193.0,35,0.006253,474,0.2,1.0,...,70.0,0.0,0.1,NaN,2.0,NaN,NaN,7.3,1.3,2.6
4,68.0,43893.0,13.3,0,193.0,35,0.006253,474,0.2,1.0,...,70.0,0.0,0.1,NaN,2.0,NaN,NaN,7.3,1.3,2.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95843,60.0,43922.0,14.9,NaN,263.0,NaN,0.032218,NaN,NaN,22.0,...,1291.0,245.0,2.2,NaN,NaN,NaN,NaN,NaN,1.3,5.2
95844,55.0,43936.0,14.0,NaN,298.0,NaN,0.032218,NaN,NaN,22.0,...,1291.0,245.0,2.2,NaN,NaN,NaN,NaN,NaN,1.3,5.2
95845,68.0,43950.0,12.6,NaN,190.0,NaN,0.032218,NaN,NaN,22.0,...,1291.0,245.0,2.2,NaN,NaN,NaN,NaN,NaN,1.3,5.2
95846,69.0,43964.0,12.4,NaN,148.0,NaN,0.032218,NaN,NaN,22.0,...,1291.0,245.0,2.2,NaN,NaN,NaN,NaN,NaN,1.3,5.2


In [14]:
df_weather = pd.read_csv("/kaggle/input/autonomous-greenhouse-challengeagc-2nd-2019/Weather/Weather.csv")
# give me "%time" i give weather

,%time,AbsHumOut,Iglob,PARout,Pyrgeo,RadSum,Rain,Rhout,Tout,Winddir,Windsp
0,43815.00000,6.220954,0.0,0.0,-72.0,215.0,0.0,80.6,6.9,32.0,4.7
1,43815.00347,6.220954,0.0,0.0,-73.0,0.0,0.0,80.6,6.9,32.0,4.7
2,43815.00694,6.205565,0.0,0.0,-76.0,0.0,0.0,80.4,6.9,32.0,4.7
3,43815.01042,6.190173,0.0,0.0,-77.0,0.0,0.0,80.2,6.9,32.0,4.7
4,43815.01389,6.162624,0.0,0.0,-75.0,0.0,0.0,80.9,6.7,32.0,4.7
